# Model

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import *
from keras.layers import *
from keras.optimizers import *
import keras

Using TensorFlow backend.


In [2]:
filepath="./"

In [3]:
X = np.load(filepath + "X_train.npy")
y = np.load(filepath + "y_train.npy")

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [5]:
mean = X_train.mean()
std = X_train.std()

In [6]:
X_train = (X_train-mean)/std
X_test = (X_test-mean)/std

In [7]:
uniq_X = np.unique(X_train)

In [8]:
print("len(np.unique): "+str(len(uniq_X)))
print("np.unique: ", uniq_X)

len(np.unique): 255
np.unique:  [-0.90075225 -0.87799746 -0.8552427  -0.83248794 -0.8097332  -0.7869784
 -0.76422364 -0.7414689  -0.7187141  -0.6959594  -0.6732046  -0.6504498
 -0.6276951  -0.6049403  -0.58218557 -0.5594308  -0.53667605 -0.51392126
 -0.4911665  -0.46841174 -0.44565696 -0.4229022  -0.40014744 -0.37739268
 -0.35463792 -0.33188316 -0.3091284  -0.28637362 -0.26361886 -0.2408641
 -0.21810934 -0.19535457 -0.1725998  -0.14984505 -0.12709028 -0.10433552
 -0.08158075 -0.05882599 -0.03607123 -0.01331646  0.0094383   0.03219306
  0.05494783  0.07770259  0.10045736  0.12321211  0.14596689  0.16872165
  0.1914764   0.21423118  0.23698594  0.2597407   0.28249547  0.30525023
  0.328005    0.35075974  0.3735145   0.3962693   0.41902405  0.4417788
  0.46453357  0.48728833  0.5100431   0.5327979   0.5555526   0.5783074
  0.6010622   0.6238169   0.6465717   0.6693264   0.6920812   0.71483594
  0.73759073  0.7603455   0.78310025  0.80585504  0.82860976  0.85136455
  0.87411934  0.89687407

In [9]:
del X, y

In [14]:
def unet_model(input_size = (216,320,1)):
    inputs = Input(input_size)
    
    # Contraction path
    
    conv11 = Conv2D(64, 3, activation = 'relu', padding = 'same')(inputs)
    conv12 = Conv2D(64, 3, activation = 'relu', padding = 'same')(conv11)
    
    pool1 = MaxPooling2D(pool_size=(2,2), strides = 2)(conv12)
    
    conv21 = Conv2D(128, 3, activation = 'relu', padding = 'same')(pool1)
    conv22 = Conv2D(128, 3, activation = 'relu', padding = 'same')(conv21)
    
    pool2 = MaxPooling2D(pool_size=(2,2), strides = 2)(conv22)
    
    conv31 = Conv2D(256, 3, activation = 'relu', padding = 'same')(pool2)
    conv32 = Conv2D(256, 3, activation = 'relu', padding = 'same')(conv31)
    
    pool3 = MaxPooling2D(pool_size=(2,2), strides = 2)(conv32)
    
    conv41 = Conv2D(512, 3, activation = 'relu', padding = 'same')(pool3)
    conv42 = Conv2D(512, 3, activation = 'relu', padding = 'same')(conv41)
    
    pool4 = MaxPooling2D(pool_size=(2,2), strides = 2)(conv42)
    
    conv51 = Conv2D(1024, 3, activation = 'relu', padding = 'same')(pool4)
    conv52 = Conv2D(1024, 3, activation = 'relu', padding = 'same')(conv51)
    
    #Expansion path
    
    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same')(UpSampling2D(size=(2,2))(conv52))
    merge6 = concatenate([conv42, up6], axis = 3)
    
    conv61 = Conv2D(512, 3, activation = 'relu', padding = 'same')(merge6)
    conv62 = Conv2D(512, 3, activation = 'relu', padding = 'same')(conv61)
    
    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same')(UpSampling2D(size = (2,2))(conv62))
    merge7 = concatenate([conv32, up7], axis = 3)
    
    conv71 = Conv2D(256, 3, activation = 'relu', padding = 'same')(merge7)
    conv72 = Conv2D(256, 3, activation = 'relu', padding = 'same')(conv71)
    
    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same')(UpSampling2D(size = (2,2))(conv72))
    merge8 = concatenate([conv22, up8], axis = 3)
    
    conv81 = Conv2D(128, 3, activation = 'relu', padding = 'same')(merge8)
    conv82 = Conv2D(128, 3, activation = 'relu', padding = 'same')(conv81)
    
    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same')(UpSampling2D(size=(2,2))(conv82))
    merge9 = concatenate([conv12, up9], axis = 3)
    
    conv91 = Conv2D(64, 3, activation = 'relu', padding = 'same')(merge9)
    conv92 = Conv2D(64, 3, activation = 'relu', padding = 'same')(conv91)
    conv93 = Conv2D(2, 3, activation = 'relu', padding = 'same')(conv92)
    
    conv10 = Conv2D(1,1, activation = 'sigmoid')(conv93)
    
    model = Model(input = inputs, output = conv10)
    
    #model.compile(optimizer = Adam(lr = 1e-2), loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

In [15]:
model = unet_model()

ValueError: A `Concatenate` layer requires inputs with matching shapes except for the concat axis. Got inputs shapes: [(None, 27, 40, 512), (None, 26, 40, 512)]

In [12]:
model.compile(optimizer = Adam(lr = 1e-2), loss = 'binary_crossentropy', metrics = ['accuracy'])

In [13]:
model.fit(X_train, y_train, epochs = 1)

ValueError: Error when checking input: expected input_1 to have shape (432, 640, 1) but got array with shape (216, 320, 1)

In [1]:
model.evaluate(X_test, y_test)

NameError: name 'model' is not defined